In [60]:
# Importa las librerias a usar
from pathlib import Path
import pandas as pd
import re
from datetime import datetime
import numpy as np
from datetime import datetime, timedelta

In [2]:
# Definir la ruta de la carpeta en el escritorio
folder = Path('D:/AÑO 2024/REPORTES 37/JULIO')

# Obtener el nombre del mes de la carpeta
mes = folder.name.upper()

# Obtener el año de la carpeta anterior
año = folder.parent.parent.name.split()[-1]  # Asume que el año es la última palabra en el nombre de la carpeta

# Lista para almacenar todos los dataframes
all_dataframes = []

# Iterar sobre todos los archivos xlsx en la carpeta
for file in folder.glob('*.xlsx'):
    # Extraer el número del nombre del archivo
    file_number = re.search(r'\d+', file.stem)
    if file_number:
        file_number = file_number.group()
    else:
        file_number = 'N/A'  # Si no se encuentra un número

    try:
        # Leer específicamente la hoja "REPORTE37" del archivo Excel
        df = pd.read_excel(file, sheet_name='REPORTE37')
        
        # Añadir una nueva columna con el número del archivo
        df['day'] = file_number
        df['month'] = mes
        df['year'] = año
        
        # Añadir el dataframe a la lista
        all_dataframes.append(df)
        print(f"Archivo leído con éxito: {file.name}")
    except Exception as e:
        print(f"No se pudo leer la hoja 'REPORTE37' en el archivo {file.name}: {e}")

# Combinar todos los DataFrames en uno solo
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"Se han combinado {len(all_dataframes)} archivos en un DataFrame de {combined_df.shape[0]} filas y {combined_df.shape[1]} columnas.")
else:
    print("No se pudo leer ningún archivo correctamente.")


Archivo leído con éxito: Informe Morosidad ACR 10 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 11 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 12 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 15 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 16 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 17 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 18 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 19 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 2 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 22 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 23 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 24 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 25 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 26 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 29 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 3 JULIO.xlsx
Archivo leído con éxito: I

In [3]:
combined_df_copy = combined_df.copy()

In [5]:
# Diccionario para mapear los nombres de los meses a números
meses_a_numeros = {
    "ENERO": 1,
    "FEBRERO": 2,
    "MARZO": 3,
    "ABRIL": 4,
    "MAYO": 5,
    "JUNIO": 6,
    "JULIO": 7,
    "AGOSTO": 8,
    "SEPTIEMBRE": 9,
    "OCTUBRE": 10,
    "NOVIEMBRE": 11,
    "DICIEMBRE": 12
}

# Crear una nueva columna 'Mes_numero' con los valores numéricos de los meses
combined_df_copy['month'] = combined_df_copy['month'].map(meses_a_numeros)
combined_df_copy['year'] = combined_df_copy['year'].astype(int)
combined_df_copy['day'] = pd.to_numeric(combined_df_copy['day'])
# Crear la nueva columna 'Fecha'
combined_df_copy['Fecha'] = pd.to_datetime(combined_df_copy[['year', 'month', 'day']])
# Calcular la mora en días
combined_df_copy['Dias_mora_calculado'] = (combined_df_copy['Fecha'] - combined_df_copy['Fch Vencim Cuota'] ).dt.days
# Definir los rangos y los valores correspondientes
condiciones = [
    (combined_df_copy['Dias_mora_calculado'] < 6),
    (combined_df_copy['Dias_mora_calculado'] < 11),
    (combined_df_copy['Dias_mora_calculado'] < 31),
    (combined_df_copy['Dias_mora_calculado'] < 61),
    (combined_df_copy['Dias_mora_calculado'] < 91),
    (combined_df_copy['Dias_mora_calculado'] < 121),
    (combined_df_copy['Dias_mora_calculado'] < 361)
]

valores = [
    0,
    (combined_df_copy['Capital Vencido'] ) * 0.071,
    (combined_df_copy['Capital Vencido'] ) * 0.095,
    (combined_df_copy['Capital Vencido']) * 0.119,
    (combined_df_copy['Capital Vencido'] ) * 0.155,
    (combined_df_copy['Capital Vencido'] ) * 0.167,
    (combined_df_copy['Capital Vencido'] ) * 0.238
]

# Aplicar las condiciones para calcular 'Gastos_de_cobranza_calculados'
combined_df_copy['Gastos_de_cobranza_calculados'] = np.select(condiciones, valores, default=(combined_df_copy['Capital Vencido']) * 0.298)
# Redondear a dos decimales
combined_df_copy['Gastos_de_cobranza_calculados'] = combined_df_copy['Gastos_de_cobranza_calculados'].round(2)


In [6]:
combined_df_copy.head()

,No Crédito,Cedula,Apellidos,Nombres,Direccion,Barrio,CIUDAD1,Tel. Fijo,Celular,Cupo,...,Intereses Vencidos,Dias Mora,Valor Gastos de Cobranza,day,month,year,mont,Fecha,Dias_mora_calculado,Gastos_de_cobranza_calculados
0,1100010,1007348023,SEPULVEDA LOTERO ...,YEFERSON ...,ALEJO-marles@hotmail.com,VARIANTE,Cartago,2095000,3147727566,1,...,1506.57,1112,5458.55540,10,7,2024,7,2024-07-10,1086,5458.56
1,1100010,1007348023,SEPULVEDA LOTERO ...,YEFERSON ...,ALEJO-marles@hotmail.com,VARIANTE,Cartago,2095000,3147727566,1,...,1141.01,1081,5567.49228,10,7,2024,7,2024-07-10,1055,5567.49
2,1100010,1007348023,SEPULVEDA LOTERO ...,YEFERSON ...,ALEJO-marles@hotmail.com,VARIANTE,Cartago,2095000,3147727566,1,...,768.18,1050,5678.59562,10,7,2024,7,2024-07-10,1024,5678.60
3,1100010,1007348023,SEPULVEDA LOTERO ...,YEFERSON ...,ALEJO-marles@hotmail.com,VARIANTE,Cartago,2095000,3147727566,1,...,387.90,1020,5791.93396,10,7,2024,7,2024-07-10,994,5791.93
4,1100026,1235048274,AGUILAR RODRIGUEZ ...,RAMON ENRIQUE ...,CLL 110 # 91 308,NaN,Barranquilla,NaN,3164529173,1,...,0.00,1354,528.29738,10,7,2024,7,2024-07-10,1328,528.30


In [7]:
# Primero, agrupamos por 'No Crédito' para obtener valores únicos de 'Valor Compra' e 'Intereses de Mora'
df_credito = combined_df_copy.groupby(['Fecha','No Crédito']).agg({
    'Cedula': 'first',
    'Valor Compra': 'first',
    'Intereses de Mora': 'first',
    'Vlr. Cuota Vencida': 'sum',
    'Intereses Vencidos': 'sum',
    'Valor Gastos de Cobranza': 'sum',
    'Capital Vencido': 'sum',
    'Gastos_de_cobranza_calculados': 'sum'  
    
}).reset_index()




In [26]:
df_credito

,Fecha,No Crédito,Cedula,Valor Compra,Intereses de Mora,Vlr. Cuota Vencida,Intereses Vencidos,Valor Gastos de Cobranza,Capital Vencido,Gastos_de_cobranza_calculados
0,2024-07-02,149970,1112218564,209900,0.00,59885.14,765.67,16165.924860,54248.07,16165.92
1,2024-07-02,149974,1112218564,65000,0.00,18687.50,379.90,5006.122860,16799.07,5006.12
2,2024-07-02,150993,1113649561,189990,977.27,108570.52,2632.92,28941.444120,97118.94,28941.45
3,2024-07-02,151409,1112218564,92900,1464.36,80126.25,3209.99,20993.545720,70448.14,20993.55
4,2024-07-02,151927,1113623261,128000,0.00,59271.91,748.10,16554.841680,55553.16,16554.84
...,...,...,...,...,...,...,...,...,...,...
584711,2024-07-31,1269359,1038814514,700000,298.96,75865.45,7589.10,4921.806680,51699.65,4911.47
584712,2024-07-31,1269383,1081153015,680000,3931.92,703475.43,7372.35,64736.000000,680000.00,64600.00
584713,2024-07-31,1269389,1089512458,900000,384.32,97541.28,9757.50,6328.027776,66470.88,6314.73
584714,2024-07-31,1269392,79996167,1336000,786.56,182151.68,14484.45,12949.999832,136029.41,12922.79


In [8]:
df_credito_sum = df_credito.groupby(['Fecha','Cedula']).agg({
    'Valor Compra': 'sum',
    'Capital Vencido': 'sum',
    'Vlr. Cuota Vencida': 'sum',
    'Intereses Vencidos': 'sum',
    'Intereses de Mora': 'sum',
    'Gastos_de_cobranza_calculados': 'sum'
}).reset_index()


In [99]:
df_credito_sum

,Fecha,Cedula,Valor Compra,Intereses de Mora,Vlr. Cuota Vencida,Intereses Vencidos,Valor Gastos de Cobranza,Capital Vencido,Gastos_de_cobranza_calculados
0,2024-07-02,334993,800000,851.03,164495.22,19712.00,14974.77198,125838.42,11954.65
1,2024-07-02,521570,1805000,638452.64,1935682.10,155420.86,447198.53952,1500666.24,439587.84
2,2024-07-02,546127,404600,558.30,98919.83,6783.39,9824.05809,82555.11,7842.74
3,2024-07-02,582095,975000,328647.83,1407123.24,188233.80,268890.82980,975000.00,263834.89
4,2024-07-02,629770,217000,89225.55,173487.16,6064.80,46766.64788,156935.06,46766.64
...,...,...,...,...,...,...,...,...,...
383428,2024-07-31,1243139989,1041500,561330.61,819324.13,47185.10,199939.50314,670937.93,199939.50
383429,2024-07-31,2000001098,500000,69677.00,578822.77,33322.77,149000.00000,500000.00,149000.00
383430,2024-07-31,2000002521,840000,2315.52,205477.02,26268.14,14910.35133,139424.80,14896.22
383431,2024-07-31,2000004082,1125000,76290.15,866928.57,65950.19,141933.91167,654274.56,141933.93


# Importamos Distribucion de recuados generada de la BD

In [40]:
distribucion = pd.read_csv('C:/Users/aleja/Desktop/DISTRIBUCION_PAGOS_JULIO.csv', sep=';', decimal=',')

In [41]:
distribucion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30928 entries, 0 to 30927
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Nº DOCUMENTO       30928 non-null  int64  
 1   Nº CREDITO         30928 non-null  int64  
 2   FECHA              30928 non-null  int64  
 3   ACTION             30928 non-null  object 
 4   VALOR_PAGADO       30928 non-null  float64
 5   CAPITAL            30928 non-null  float64
 6   INTERESES          30928 non-null  float64
 7   SANCION            30928 non-null  float64
 8   SEGURO             30928 non-null  float64
 9   INTERESES DE MORA  30928 non-null  float64
dtypes: float64(6), int64(3), object(1)
memory usage: 2.4+ MB


In [42]:
distribucion['FECHA'] = pd.to_datetime(distribucion['FECHA'], format='%Y%m%d')


In [43]:
distribucion.head()

,Nº DOCUMENTO,Nº CREDITO,FECHA,ACTION,VALOR_PAGADO,CAPITAL,INTERESES,SANCION,SEGURO,INTERESES DE MORA
0,1128456503,1121574,2024-07-01,E,40922.93,13847.49,0.00,0.0,2050.50,25024.94
1,1128456503,1122833,2024-07-01,E,23697.51,6027.30,488.11,0.0,1067.07,16115.03
2,1128456503,1135602,2024-07-01,E,63661.89,0.00,0.00,0.0,0.00,63661.89
3,34613611,1230087,2024-07-01,E,160668.63,134002.08,0.00,0.0,9221.31,17445.24
4,34613611,1230087,2024-07-01,E,176055.14,138380.14,4521.60,0.0,33153.40,0.00


In [44]:
distri_agrupado = distribucion.groupby(['FECHA', 'Nº CREDITO']).agg({
    'Nº DOCUMENTO': 'first',
    'VALOR_PAGADO': 'sum',
    'CAPITAL': 'sum',
    'INTERESES': 'sum',
    'SANCION': 'sum',
    'SEGURO': 'sum',
    'INTERESES DE MORA': 'sum'  
    
}).reset_index()

In [45]:
distri_agrupado

,FECHA,Nº CREDITO,Nº DOCUMENTO,VALOR_PAGADO,CAPITAL,INTERESES,SANCION,SEGURO,INTERESES DE MORA
0,2024-07-01,1121574,1128456503,45130.58,13847.49,0.00,0.0,2050.50,29232.59
1,2024-07-01,1122833,1128456503,25528.96,6027.30,488.11,0.0,1067.07,17946.48
2,2024-07-01,1135602,1128456503,69597.03,0.00,0.00,0.0,0.00,69597.03
3,2024-07-01,1136363,1128456503,3510.48,0.00,0.00,0.0,0.00,3510.48
4,2024-07-01,1143802,1128456503,6232.95,0.00,0.00,0.0,0.00,6232.95
...,...,...,...,...,...,...,...,...,...
19766,2024-07-30,1269121,1081153985,134302.14,102607.61,8456.58,0.0,15392.65,7845.30
19767,2024-07-30,1269196,1005980362,161000.00,102863.53,17165.24,0.0,33082.36,7888.87
19768,2024-07-30,1269197,66772591,17848.50,14569.27,739.67,0.0,1425.60,1113.96
19769,2024-07-30,1269296,16256498,227081.01,197854.94,4625.76,0.0,9472.40,15127.91


In [46]:
distri_agrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19771 entries, 0 to 19770
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FECHA              19771 non-null  datetime64[ns]
 1   Nº CREDITO         19771 non-null  int64         
 2   Nº DOCUMENTO       19771 non-null  int64         
 3   VALOR_PAGADO       19771 non-null  float64       
 4   CAPITAL            19771 non-null  float64       
 5   INTERESES          19771 non-null  float64       
 6   SANCION            19771 non-null  float64       
 7   SEGURO             19771 non-null  float64       
 8   INTERESES DE MORA  19771 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 1.4 MB


In [47]:
df_credito.columns

Index(['Fecha', 'No Crédito', 'Cedula', 'Valor Compra', 'Intereses de Mora',
       'Vlr. Cuota Vencida', 'Intereses Vencidos', 'Valor Gastos de Cobranza',
       'Capital Vencido', 'Gastos_de_cobranza_calculados'],
      dtype='object')

In [48]:
df_credito.shape

(584716, 10)

In [49]:
distri_agrupado.shape

(19771, 9)

In [50]:
distri_agrupado.columns

Index(['FECHA', 'Nº CREDITO', 'Nº DOCUMENTO', 'VALOR_PAGADO', 'CAPITAL',
       'INTERESES', 'SANCION', 'SEGURO', 'INTERESES DE MORA'],
      dtype='object')

In [90]:
df_merge = pd.merge(distri_agrupado, df_credito, how='outer', left_on=['FECHA', 'Nº CREDITO'], right_on=['Fecha','No Crédito'])

In [94]:
df_merge = pd.merge(distri_agrupado, df_credito, how='outer', left_on=['FECHA', 'Nº CREDITO'], right_on=['Fecha','No Crédito'])

# Llenar los valores NaN en 'FECHA' y 'Nº CREDITO' con los valores de 'Fecha' y 'No Crédito'
df_merge['FECHA'] = df_merge['FECHA'].fillna(df_merge['Fecha'])
df_merge['Nº CREDITO'] = df_merge['Nº CREDITO'].fillna(df_merge['No Crédito'])
df_merge['Nº DOCUMENTO'] = df_merge['Nº DOCUMENTO'].fillna(df_merge['Cedula'])

In [95]:
df_merge

,FECHA,Nº CREDITO,Nº DOCUMENTO,VALOR_PAGADO,CAPITAL,INTERESES,SANCION,SEGURO,INTERESES DE MORA,Fecha,...,Cedula,Valor Compra,Intereses de Mora,Vlr. Cuota Vencida,Intereses Vencidos,Valor Gastos de Cobranza,Capital Vencido,Gastos_de_cobranza_calculados,dia_semana,dia_semana_num
0,2024-07-01,1121574.0,1.128457e+09,45130.58,13847.49,0.00,0.0,2050.50,29232.59,2024-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-07-01,1122833.0,1.128457e+09,25528.96,6027.30,488.11,0.0,1067.07,17946.48,2024-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-07-01,1135602.0,1.128457e+09,69597.03,0.00,0.00,0.0,0.00,69597.03,2024-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-07-01,1136363.0,1.128457e+09,3510.48,0.00,0.00,0.0,0.00,3510.48,2024-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-07-01,1143802.0,1.128457e+09,6232.95,0.00,0.00,0.0,0.00,6232.95,2024-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598469,2024-07-31,1269359.0,1.038815e+09,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-31,...,1.038815e+09,700000.0,298.96,75865.45,7589.10,4921.806680,51699.65,4911.47,Wednesday,2.0
598470,2024-07-31,1269383.0,1.081153e+09,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-31,...,1.081153e+09,680000.0,3931.92,703475.43,7372.35,64736.000000,680000.00,64600.00,Wednesday,2.0
598471,2024-07-31,1269389.0,1.089512e+09,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-31,...,1.089512e+09,900000.0,384.32,97541.28,9757.50,6328.027776,66470.88,6314.73,Wednesday,2.0
598472,2024-07-31,1269392.0,7.999617e+07,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-31,...,7.999617e+07,1336000.0,786.56,182151.68,14484.45,12949.999832,136029.41,12922.79,Wednesday,2.0


In [53]:
merge.to_excel('../files/distribucion_merge.xlsx', sheet_name='DISTRIBUCION_PRUEBA1', index=False)

# Calcular diferencia

In [66]:
festivos_colombia = ['2024-03-25','2024-03-28','2024-03-29','2024-05-01','2024-05-13','2024-06-03','2024-06-10','2024-07-01','2024-07-20','2024-08-07',
                     '2024-08-19','2024-10-14','2024-11-04','2024-11-11','2024-12-08','2024-12-25'

                    ]

# Convertimos las fechas de festivos a objetos datetime
festivos_colombia = [datetime.strptime(fecha, '%Y-%m-%d').date() for fecha in festivos_colombia]

In [55]:
# Crea una nueva columna con el día de la semana
df_credito['dia_semana'] = df_credito['Fecha'].dt.day_name()

# O si prefieres números (0 = Lunes, 6 = Domingo)
df_credito['dia_semana_num'] = df_credito['Fecha'].dt.dayofweek

In [58]:
distri_agrupado.columns

Index(['FECHA', 'Nº CREDITO', 'Nº DOCUMENTO', 'VALOR_PAGADO', 'CAPITAL',
       'INTERESES', 'SANCION', 'SEGURO', 'INTERESES DE MORA'],
      dtype='object')

In [59]:
df_credito.columns


Index(['Fecha', 'No Crédito', 'Cedula', 'Valor Compra', 'Intereses de Mora',
       'Vlr. Cuota Vencida', 'Intereses Vencidos', 'Valor Gastos de Cobranza',
       'Capital Vencido', 'Gastos_de_cobranza_calculados', 'dia_semana',
       'dia_semana_num'],
      dtype='object')

In [76]:
import pandas as pd
from datetime import datetime, timedelta

def obtener_dia_anterior_habil(fecha, festivos_colombia):
    dia = fecha
    while True:
        dia -= timedelta(days=1)
        if dia.weekday() < 5 and dia not in festivos_colombia:
            return dia

def calcular_diferencias(df_credito, distri_agrupado, festivos_colombia):
    # Aseguramos que la columna 'Fecha' en df_credito y 'FECHA' en distri_agrupado sean de tipo datetime
    df_credito['Fecha'] = pd.to_datetime(df_credito['Fecha'])
    distri_agrupado['FECHA'] = pd.to_datetime(distri_agrupado['FECHA'])
    
    # Ordenamos el DataFrame df_credito por 'No Crédito' y 'Fecha'
    df_credito = df_credito.sort_values(['No Crédito', 'Fecha'])
    
    # Creamos columnas para almacenar las diferencias
    columnas_a_restar = ['Intereses de Mora', 'Vlr. Cuota Vencida', 'Intereses Vencidos', 
                         'Valor Gastos de Cobranza', 'Capital Vencido', 'Gastos_de_cobranza_calculados']
    
    for col in columnas_a_restar:
        df_credito[f'Diff_{col}'] = 0.0
    
    # Iteramos sobre cada pago en distri_agrupado
    for _, pago in distri_agrupado.iterrows():
        credito = pago['Nº CREDITO']
        fecha_pago = pago['FECHA']
        
        # Filtramos las filas de df_credito que corresponden al mismo crédito y fecha de pago
        grupo_credito = df_credito[df_credito['No Crédito'] == credito]
        fechas = grupo_credito['Fecha'].tolist()
        
        if fecha_pago in fechas:
            idx_actual = grupo_credito[grupo_credito['Fecha'] == fecha_pago].index[0]
            fecha_anterior = obtener_dia_anterior_habil(fecha_pago, festivos_colombia)
            filas_anteriores = grupo_credito[grupo_credito['Fecha'] == fecha_anterior]
            
            if not filas_anteriores.empty:
                idx_anterior = filas_anteriores.index[0]
                
                # Calculamos las diferencias
                for col in columnas_a_restar:
                    diff = df_credito.loc[idx_actual, col] - df_credito.loc[idx_anterior, col]
                    df_credito.loc[idx_actual, f'Diff_{col}'] = diff
            else:
                print(f"No se encontró fecha anterior hábil para el crédito {credito} en la fecha {fecha_pago}")
        else:
            print(f"No se encontró la fecha de pago {fecha_pago} para el crédito {credito} en df_credito")
    
    return df_credito

# Aplicamos la función a nuestro DataFrame
df_credito_con_diferencias = calcular_diferencias(df_credito, distri_agrupado, festivos_colombia)


No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1121574 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1122833 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1135602 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1136363 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1143802 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1230087 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1236506 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1238674 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1239116 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1240970 en df_credito
No se encontró la fecha de pago 2024-07-01 00:00:00 para el crédito 1242434 en df_credito
No se enco

In [ ]:
def obtener_dia_anterior_habil(fecha, festivos_colombia):
    dia = fecha
    while True:
        dia -= timedelta(days=1)
        if dia.weekday() < 5 and dia not in festivos_colombia:
            return dia

def calcular_diferencias(df_merge, festivos_colombia):   
    # Ordenamos el DataFrame por 'Nº CREDITO' y 'Fecha'
    df_merge = df_merge.sort_values(['Fecha','Nº CREDITO'])
    
    # Creamos columnas para almacenar las diferencias
    columnas_a_restar = ['Intereses de Mora', 'Vlr. Cuota Vencida', 'Intereses Vencidos', 
                         'Valor Gastos de Cobranza', 'Capital Vencido', 'Gastos_de_cobranza_calculados']
    
    for col in columnas_a_restar:
        df_merge[f'Diff_{col}'] = 0.0
    
    # Iteramos sobre cada grupo de 'Nº CREDITO'
    for credito, grupo in df_merge.groupby('Nº CREDITO'):
        fechas = grupo['Fecha'].tolist()
        for i in range(1, len(fechas)):
            fecha_actual = fechas[i]
            fecha_anterior = obtener_dia_anterior_habil(fecha_actual, festivos_colombia)
            
            # Obtenemos los índices de las filas correspondientes
            idx_actual = grupo[grupo['Fecha'] == fecha_actual].index[0]
            filas_anteriores = grupo[grupo['Fecha'] == fecha_anterior]
            
            if not filas_anteriores.empty:
                idx_anterior = filas_anteriores.index[0]
                
                # Calculamos las diferencias
                for col in columnas_a_restar:
                    diff = df_merge.loc[idx_actual, col] - df_merge.loc[idx_anterior, col]
                    df_merge.loc[idx_actual, f'Diff_{col}'] = diff
            else:
                print(f"No se encontró fecha anterior hábil para el crédito {credito} en la fecha {fecha_actual}")
    
    return df_merge


# Aplicamos la función a nuestro DataFrame
df_credito_con_diferencias = calcular_diferencias(df_merge, festivos_colombia)


In [101]:
# Ordenamos el DataFrame por 'Nº CREDITO' y 'Fecha'
df_merge = df_merge.sort_values(['Fecha','Nº CREDITO'])

In [102]:
columnas_a_restar = ['Intereses de Mora', 'Vlr. Cuota Vencida', 'Intereses Vencidos', 
                         'Valor Gastos de Cobranza', 'Capital Vencido', 'Gastos_de_cobranza_calculados']

In [100]:
for col in columnas_a_restar:
        df_merge[f'Diff_{col}'] = 0.0

In [107]:
# Iteramos sobre cada grupo de 'Nº CREDITO'
for credito, grupo in df_merge.groupby(['Fecha','No Crédito']):
    fechas = grupo['Fecha'].tolist()
    for i in range(1, len(fechas)):
        fecha_actual = fechas[i]
        print('Fecha Actual', fechas[i])
        fecha_anterior = obtener_dia_anterior_habil(fecha_actual, festivos_colombia)
        print('fecha Aneterior',fecha_anterior)
        
        # Obtenemos los índices de las filas correspondientes
        idx_actual = grupo[grupo['Fecha'] == fecha_actual].index[0]
        print('index actual',idx_actual)
        filas_anteriores = grupo[grupo['Fecha'] == fecha_anterior]
        print('fila anterior', filas_anteriores)
        
        if not filas_anteriores.empty:
            idx_anterior = filas_anteriores.index[0]
            
            # Calculamos las diferencias
            for col in columnas_a_restar:
                diff = df_merge.loc[idx_actual, col] - df_merge.loc[idx_anterior, col]
                df_merge.loc[idx_actual, f'Diff_{col}'] = diff
        else:
            print(f"No se encontró fecha anterior hábil para el crédito {credito} en la fecha {fecha_actual}")

In [113]:
df_merge[(df_merge['Cedula'] == 1128456503) & (df_merge['Fecha'] == '2024-07-30')]


,FECHA,Nº CREDITO,Nº DOCUMENTO,VALOR_PAGADO,CAPITAL,INTERESES,SANCION,SEGURO,INTERESES DE MORA,Fecha,...,Capital Vencido,Gastos_de_cobranza_calculados,dia_semana,dia_semana_num,Diff_Intereses de Mora,Diff_Vlr. Cuota Vencida,Diff_Intereses Vencidos,Diff_Valor Gastos de Cobranza,Diff_Capital Vencido,Diff_Gastos_de_cobranza_calculados
548400,2024-07-30,1121574.0,1.128457e+09,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-30,...,43197.26,12872.78,Tuesday,1.0,0.0,0.0,0.0,0.0,0.0,0.0
548509,2024-07-30,1122833.0,1.128457e+09,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-30,...,18802.21,5603.06,Tuesday,1.0,0.0,0.0,0.0,0.0,0.0,0.0
549643,2024-07-30,1135602.0,1.128457e+09,116446.87,19538.74,3674.87,0.0,4486.3,88746.96,2024-07-30,...,190079.29,56643.63,Tuesday,1.0,0.0,0.0,0.0,0.0,0.0,0.0
549713,2024-07-30,1136363.0,1.128457e+09,33553.13,0.00,0.00,0.0,0.0,33553.13,2024-07-30,...,112428.24,33503.60,Tuesday,1.0,0.0,0.0,0.0,0.0,0.0,0.0
550373,2024-07-30,1143802.0,1.128457e+09,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-30,...,175650.39,52343.82,Tuesday,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

    
    # Creamos columnas para almacenar las diferencias
    
    
    
    
    